### IMPORTS

In [1]:
import numpy as np

In [2]:
from sklearn.linear_model import LinearRegression

In [3]:
import tensorflow as tf
from tensorflow.python.keras import Sequential, Input, Model
from tensorflow.python.keras.layers import Dense, LSTM, Dropout

In [4]:
import ipynb.fs.full.splitting as splitting

In [5]:
from tcn import TCN, tcn_full_summary

### LINEAR REGRESSION

In [6]:
def linear_regression(data, settings):
    
    # INSTANTIATE MODEL
    linear = LinearRegression()
    
    # TRAIN WITH TRAIN DATA
    model = linear.fit(data['train']['features'], data['train']['labels'])
    
    # PREDICT USING TEST DATA
    predictions = model.predict(data['test']['features'])
    
    return np.ndarray.flatten(predictions)

### LONG-SHORT TERM MEMORY

In [7]:
def add_lstm_layer(name, settings, model, index, shape):
    
    # AVAILABLE LSTM LAYERS
    available = {
        'lstm': LSTM,
        'dropout': Dropout,
        'dense': Dense
    }

    # SELECT THE CORRECT FUNCTION
    func = available[name]

    # INJECT INPUT LAYER DIMENSIONS TO SETTINGS
    if index == 0:
        settings['input_shape'] = (shape[1], shape[2])
        model.add(func(**settings))

    # OTHERWISE, DEFAULT TO BASE SETTINGS
    else:
        model.add(func(**settings))

In [31]:
def long_short_term(data, settings):
    
    # INSTANTIATE MODEL
    model = Sequential()
    
    # TRAIN DATA GENERATOR
    train_generator = splitting.generator(
        data['train'],
        settings['morph'],
        shuffle=True
    )
    
    # LOOP THROUGH LAYERS
    for index, layer in enumerate(settings['layers']):
        
        # LAYER PROPS
        name = list(layer)[0]
        params = layer[name]
        
        # ADD LAYER
        add_lstm_layer(
            name,
            params,
            model,
            index,
            train_generator[0][0].shape
        )
    
    # COMPILE THE MODEL
    model.compile(
        loss=settings['loss'],
        optimizer=settings['optimizer']
    )
    
    # FIT USING THE TRAIN GENERATOR
    model.fit_generator(
        train_generator,
        steps_per_epoch=len(train_generator),
        epochs=settings['epochs'],
        verbose=0
    )
    
    # TEST DATA GENERATOR
    test_generator = splitting.generator(
        data['test'],
        settings['morph'],
        shuffle=False
    )
    
    # PREDICT USING TEST DATA
    predictions = model.predict(test_generator)
    
    return np.ndarray.flatten(predictions)

### TEMPORAL CONVOLUTIONAL NETWORK

In [53]:
def add_tcn_layers(name, settings, index, model_input, old_output):
    
    # NEW OUTPUT PLACEHOLDER
    new_output = []
    
    # AVAILABLE TNC LAYERS
    available = {
        'tcn': TCN,
        'dense': Dense,
        'dropout': Dropout
    }

    # SELECT THE CORRECT FUNCTION
    func = available[name]

    # INJECT WITH INPUT LAYER
    if index == 0:
        new_output = func(**settings)(model_input)

    # OTHERWISE, INJECT WITH OUTPUT LAYER
    else:
        new_output = func(**settings)(old_output)
        
    return new_output

In [54]:
def temporal_convolutional_network(data, settings):

    # TRAIN DATA GENERATOR
    train_generator = splitting.generator(
        data['train'],
        settings['morph'],
        shuffle=True
    )

    # INPUT & OUTPUT LAYER
    model_input = Input(batch_shape=train_generator[0][0].shape)
    model_output = []
    
    # LOOP THROUGH REQUESTED MODEL LAYERS
    for index, layer in enumerate(settings['layers']):

        # LAYER PROPS
        name = list(layer)[0]
        params = layer[name]
        
        # CHURN MODEL LAYERS
        model_output = add_tcn_layers(
            name,
            params,
            index,
            model_input,
            model_output
        )

    #  INSTANTIATE MODEL AND ASSIGN INPUT AND OUTPUT
    model = Model(inputs=[model_input], outputs=[model_output])

    # COMPILE THE MODEL
    model.compile(
        optimizer=settings['optimizer'],
        loss=settings['loss']
    )

    #  PRINT MODEL STATS
    tcn_full_summary(model, expand_residual_blocks=False)

    # FIT THE MODEL WITH THE TRAIN GENERATOR
    model.fit_generator(
        train_generator,
        steps_per_epoch=len(train_generator),
        epochs=settings['epochs'],
        verbose=0
    )
    
    # TEST DATA GENERATOR
    test_generator = splitting.generator(
        data['test'],
        settings['morph'],
        shuffle=False
    )

    #  PREDICT USING TEST GENERATOR
    predictions = model.predict(test_generator)

    return np.ndarray.flatten(predictions)

### START TRAINING A MODEL

In [12]:
def start(dataset, name, settings):
    
    # AVAILABLE MODELS
    available = {
        'linreg': linear_regression,
        'lstm': long_short_term,
        'tcn': temporal_convolutional_network
    }
    
    # SELECT THE CORRECT FUNCTION & START
    return available[name](dataset, settings)